# CS506 Project Initial Report

## Chengyu Deng, Xiaotong Niu, Qian Zhang

This Jupyter Notebook is the code for the CS506 Initial Report. We processed the main data set and make it ready to use for analysis. 


**Part 1 Data processing**

We used Building Energy Reporting And Disclosure Ordinance (BERDO) data provided by [Analyze Boston - Boston.gov](https://data.boston.gov/) to do the analysis. 
* **_berdo2017.csv_** for the year of 2017.
* **_2016-reported-energy-and-water-metrics.xlsx_** for the year of 2016.
* **_2015-reported-energy-and-water-metrics.xlsx_** for the year of 2015.

The source of the data is: https://data.boston.gov/dataset/building-energy-reporting-and-disclosure-ordinance


In [2]:
# Import data and put them into Pandas dataframes
import csv
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize


# 2015 xlsx -> csv
data_2015_xlsx = pd.read_excel('2015-reported-energy-and-water-metrics.xlsx', index_col=None)
data_2015_xlsx.to_csv('berdo2015.csv')

# 2016 xlsx -> csv
data_2016_xlsx = pd.read_excel('2016-reported-energy-and-water-metrics.xlsx', index_col=None)
data_2016_xlsx.to_csv('berdo2016.csv')

# 2017
df_2015 = pd.read_csv('berdo2015.csv')
df_2016 = pd.read_csv('berdo2016.csv')
df_2017 = pd.read_csv('berdo2017.csv', encoding = "ISO-8859-1")


# Testing code (For debugging purpose)

# print(type(df_2015))
# print(type(df_2016))
# print(type(df_2017))
# print('-------------------------------')
# print(df_2015.shape)
# print(df_2016.shape)
# print(df_2017.shape)
# print('-------------------------------')
# print(list(df_2015.columns.values))
# print('-------------------------------')
# print(list(df_2016.columns.values))
# print('-------------------------------')
# print(list(df_2017.columns.values))
# print('-------------------------------')

In [3]:
# Data Trimming

# Dataframe columns names handling
if 'Years Reported' in df_2016:
    df_2016.drop('Years Reported', axis = 1, inplace = True)
if 'Years Reported' in df_2017:
    df_2017.drop('Years Reported', axis = 1, inplace = True)

if ' Gross Area (sq ft) ' in df_2017:
    df_2017.rename(index=str, columns={' Gross Area (sq ft) ': 'Gross Area (sq ft)'}, inplace = True)
if ' GHG Emissions (MTCO2e) ' in df_2017:
    df_2017.rename(index=str, columns={' GHG Emissions (MTCO2e) ': 'GHG Emissions (MTCO2e)'}, inplace = True)
if ' Total Site Energy (kBTU) ' in df_2017:
    df_2017.rename(index=str, columns={' Total Site Energy (kBTU) ': 'Total Site Energy (kBTU)'}, inplace = True)
if ' Onsite Renewable (kWh) ' in df_2017:
    df_2017.rename(index=str, columns={' Onsite Renewable (kWh) ': 'Onsite Renewable (kWh)'}, inplace = True)

    
# Testing code (For debugging purpose)

# print(type(df_2015))
# print(type(df_2016))
# print(type(df_2017))
# print(df_2015.shape)
# print(df_2016.shape)
# print(df_2017.shape)
# print('-------------------------------')
# print(list(df_2015.columns.values))
# print('-------------------------------')
# print(list(df_2016.columns.values))
# print('-------------------------------')
# print(list(df_2017.columns.values))
# print('-------------------------------')


# Select properties which belong to BU
with open("BU_Property_List.csv", 'r') as propertyFile:
    reader = csv.reader(propertyFile, delimiter='\t')
    propertyList_convol = list(reader)
    
    propertyList = []
    for each in propertyList_convol:
        for element in each:
            propertyList.append(element)
            
df2015_BU = df_2015.loc[df_2015['Property Name'].isin(propertyList)]
df2016_BU = df_2016.loc[df_2016['Property Name'].isin(propertyList)]
df2017_BU = df_2017.loc[df_2017['Property Name'].isin(propertyList)]

# Testing code (For debugging purpose)

print('BU properties ------------------------------------')
print(df2015_BU.shape)
print(df2016_BU.shape)
print(df2017_BU.shape)

# Drop rows of data whose Site EUI (kBTU/sf) is not available

df2015_BU = df2015_BU.loc[df_2015['Site EUI (kBTU/sf)'] != 'Not Available']
df2016_BU = df2016_BU.loc[df_2016['Site EUI (kBTU/sf)'] != 'Not Available']
df2017_BU = df2017_BU.loc[df_2017['Site EUI (kBTU/sf)'] != 'Not Available']

# Testing code (For debugging purpose)

# print(len(propertyList))
# print('------------------------------------')
# print(propertyList)
# print('------------------------------------')
# # print(df2015_BU)
# # print(df2016_BU)
# # print(df2017_BU)
print('BU properties Site EUI (kBTU/sf)] != Not Available----')
print(df2015_BU.shape)
print(df2016_BU.shape)
print(df2017_BU.shape)
print('------------------------------------')
print(list(df_2015.columns.values))
print('------------------------------------')
# print(df2015_BU['Property Name'])
# print(df2016_BU['Property Name'])
# print(df2017_BU['Property Name'])
# print('------------------------------------')
# print(df2017_BU[['Site EUI (kBTU/sf)', '% Electricity', '% Gas', '% Steam']])

BU properties ------------------------------------
(55, 23)
(56, 23)
(59, 23)
BU properties Site EUI (kBTU/sf)] != Not Available----
(37, 23)
(54, 23)
(57, 23)
------------------------------------
['Unnamed: 0', 'Property Name', 'Reported', 'Property Type', 'Address', 'ZIP', 'Gross Area (sq ft)', 'Site EUI (kBTU/sf)', 'Energy Star Score', 'Energy Star Certified', 'Property Uses', 'Year Built', 'GHG Emissions (MTCO2e)', 'GHG Intensity (kgCO2/sf)', 'Total Site Energy (kBTU)', '% Electricity', '% Gas', '% Steam', 'Water Intensity (gal/sf)', 'Onsite Solar (kWh)', 'User Submitted Info', 'User Submitted Link', 'Tax Parcel']
------------------------------------


**Part 2 Monthly energy consumption assignment**

Since the BERDO data is yearly based, we need to calculate the monthly energy consumption for each building based on the energy consumption distribution provided by Kevin Zheng from Sustainability@BU

In [4]:
# Calculate monthly energy consumption electricity/natrual gas

#[Jan, Feb, Mar, ..., Nov, Dec]
share_2015 = np.array([0.2038, 0.1062, 0.0778, 0.0620, 0.0589, 0.0588, 0.0473, 0.0813, 0.0705, 0.0732, 0.0673, 0.0929])
share_2016 = np.array([0.0919, 0.1179, 0.0995, 0.0642, 0.0710, 0.0733, 0.0660, 0.0673, 0.0733, 0.0774, 0.0959, 0.1023])
share_2017 = np.array([0.1146, 0.1031, 0.0872, 0.0862, 0.0699, 0.0650, 0.0661, 0.0635, 0.0865, 0.0671, 0.0727, 0.1181])

#          Jan                               Dec
#[[Natural Gas, Electricity], ...[Natural Gas, Electricity]]
NESplits_2015 = np.array(
    [[0.6877, 0.3123], 
     [0.6654, 0.3346], 
     [0.6079, 0.3921], 
     [0.5086, 0.4914], 
     [0.3888, 0.6112], 
     [0.3663, 0.6337], 
     [0.1913, 0.8087], 
     [0.5173, 0.4827], 
     [0.5090, 0.4910], 
     [0.5598, 0.4402], 
     [0.5245, 0.4755], 
     [0.6743, 0.3257]])

NESplits_2016 = np.array(
    [[0.6519, 0.3481], 
     [0.6802, 0.3198], 
     [0.6496, 0.3504], 
     [0.5395, 0.4605], 
     [0.4350, 0.5650], 
     [0.4154, 0.5846], 
     [0.3339, 0.6661], 
     [0.3336, 0.6664], 
     [0.4442, 0.5558], 
     [0.5321, 0.4679], 
     [0.6225, 0.3775], 
     [0.6477, 0.3523]])

NESplits_2017 = np.array(
    [[0.6739, 0.3261], 
     [0.6707, 0.3293], 
     [0.6211, 0.3789], 
     [0.5920, 0.4080], 
     [0.5074, 0.4926], 
     [0.3905, 0.6095], 
     [0.3199, 0.6801], 
     [0.3178, 0.6822],
     [0.3672, 0.6328], 
     [0.3960, 0.6040],  
     [0.5386, 0.4614], 
     [0.6525, 0.3475]])


# Testing code (For debugging purpose)
# print(np.sum(share_2015))
# print(np.sum(share_2016))
# print(np.sum(share_2017))
# print('------------------------------------')
# print(np.sum(NESplits_2015))
# print(np.sum(NESplits_2016))
# print(np.sum(NESplits_2017))
# print('------------------------------------')
# print(share_2015.shape) -> (12, )
# print(share_2016.shape) -> (12, )
# print(share_2017.shape) -> (12, )

# Change (12, ) to (12, 1) since the we can not do matrix multiplication with shape (12, ) 
share_2015 = np.reshape(share_2015, (share_2015.shape[0], 1))
share_2016 = np.reshape(share_2016, (share_2016.shape[0], 1))
share_2017 = np.reshape(share_2017, (share_2017.shape[0], 1))


# Testing code (For debugging purpose)

# print('------------------------------------')
# print(share_2015.shape)
# print(share_2016.shape)
# print(share_2017.shape)
# print('------------------------------------')
# print(NESplits_2015.shape)
# print(NESplits_2016.shape)
# print(NESplits_2017.shape)

share_2015_G_E = share_2015 * NESplits_2015
share_2016_G_E = share_2016 * NESplits_2016
share_2017_G_E = share_2017 * NESplits_2017


# Testing code (For debugging purpose)

# print('------------------------------------')
# print(share_2015_G_E.shape)
# print(share_2016_G_E.shape)
# print(share_2017_G_E.shape)
# print(share_2015_G_E)
# print(share_2016_G_E)
# print(share_2017_G_E)
# print('------------------------------------')

# print(np.sum(share_2015_G_E, axis = 1))
# print(np.sum(share_2016_G_E, axis = 1))
# print(np.sum(share_2017_G_E, axis = 1))

# Normalize the seperated distribution of electircity/natrual gas comsumption
G_dstri_2015 = share_2015_G_E[:, 0]
E_dstri_2015 = share_2015_G_E[:, 1]

G_dstri_2016 = share_2016_G_E[:, 0]
E_dstri_2016 = share_2016_G_E[:, 1]

G_dstri_2017 = share_2017_G_E[:, 0]
E_dstri_2017 = share_2017_G_E[:, 1]

# Reshape from (12, ) to (12, 1)
G_dstri_2015 = np.reshape(G_dstri_2015, (G_dstri_2015.shape[0], 1))
E_dstri_2015 = np.reshape(E_dstri_2015, (E_dstri_2015.shape[0], 1))

G_dstri_2016 = np.reshape(G_dstri_2016, (G_dstri_2016.shape[0], 1))
E_dstri_2016 = np.reshape(E_dstri_2016, (E_dstri_2016.shape[0], 1))

G_dstri_2017 = np.reshape(G_dstri_2017, (G_dstri_2017.shape[0], 1))
E_dstri_2017 = np.reshape(E_dstri_2017, (E_dstri_2017.shape[0], 1))

# Testing code (For debugging purpose)

# print('------------------------------------')
# print(share_2015_G_E)
# print(G_dstri_2015)
# print(E_dstri_2015)

# print('------------------------------------')
# print(share_2016_G_E)
# print(G_dstri_2016)
# print(E_dstri_2016)

# print('------------------------------------')
# print(share_2017_G_E)
# print(G_dstri_2017)
# print(E_dstri_2017)

G_dstri_2015_Norm = normalize(G_dstri_2015, norm='l1', axis = 0)
E_dstri_2015_Norm = normalize(E_dstri_2015, norm='l1', axis = 0)

G_dstri_2016_Norm = normalize(G_dstri_2016, norm='l1', axis = 0)
E_dstri_2016_Norm = normalize(E_dstri_2016, norm='l1', axis = 0)

G_dstri_2017_Norm = normalize(G_dstri_2017, norm='l1', axis = 0)
E_dstri_2017_Norm = normalize(E_dstri_2017, norm='l1', axis = 0)

print('------------------------------------')
print(G_dstri_2015_Norm)
print('------------------------------------')
print(E_dstri_2015_Norm)
print('------------------------------------')
print(G_dstri_2016_Norm)
print('------------------------------------')
print(E_dstri_2016_Norm)
print('------------------------------------')
print(G_dstri_2017_Norm)
print('------------------------------------')
print(E_dstri_2017_Norm)
print('------------------------------------')

print(np.sum(G_dstri_2015_Norm))
print(np.sum(E_dstri_2015_Norm))
print(np.sum(G_dstri_2016_Norm))
print(np.sum(E_dstri_2016_Norm))
print(np.sum(G_dstri_2017_Norm))
print(np.sum(E_dstri_2017_Norm))


------------------------------------
[[ 0.25027659]
 [ 0.12618983]
 [ 0.08445566]
 [ 0.05630994]
 [ 0.0408939 ]
 [ 0.03846195]
 [ 0.01615821]
 [ 0.07510175]
 [ 0.06408021]
 [ 0.07317471]
 [ 0.06303439]
 [ 0.11186286]]
------------------------------------
[[ 0.14464954]
 [ 0.08075908]
 [ 0.06932938]
 [ 0.0692417 ]
 [ 0.08181624]
 [ 0.08468411]
 [ 0.08693396]
 [ 0.08918847]
 [ 0.07867043]
 [ 0.07323219]
 [ 0.07272881]
 [ 0.06876609]]
------------------------------------
[[ 0.10994593]
 [ 0.14717468]
 [ 0.11861832]
 [ 0.0635637 ]
 [ 0.05668006]
 [ 0.05587958]
 [ 0.04044297]
 [ 0.04120252]
 [ 0.05975376]
 [ 0.07558174]
 [ 0.10955713]
 [ 0.12159962]]
------------------------------------
[[ 0.0702932 ]
 [ 0.08284877]
 [ 0.07660921]
 [ 0.06496186]
 [ 0.08814559]
 [ 0.09415786]
 [ 0.09660001]
 [ 0.0985471 ]
 [ 0.08951922]
 [ 0.07957704]
 [ 0.07954802]
 [ 0.07919214]]
------------------------------------
[[ 0.14618533]
 [ 0.13089127]
 [ 0.10251838]
 [ 0.09659456]
 [ 0.06713536]
 [ 0.0480461 ]
 

In [4]:
# One more data trim... 
'''
Since the distribution only involve with gas and electricity, we CURRENTLY drop the buildings that use steam. 
However in the future, if we get the revised consumption distribution over electricity, gas and steam, we will
re-include those dropped buildings that use steam back to analysis. 
'''

# Be careful that in 2015, 2017 data is in format of xx% while in 2016 it is 0.xx. 
# Those are all strings in dataframe, need to convert to numerical before calculation (monthly consumption assignment)

df2015_BU = df2015_BU.loc[df2015_BU['% Steam'] == '0%']
df2016_BU = df2016_BU.loc[df2016_BU['% Steam'].isnull()]
df2017_BU = df2017_BU.loc[df2017_BU['% Steam'].isnull()]



# Testing code (For debugging purpose)
print('-----2015----------------------------')
print(df2015_BU[['Site EUI (kBTU/sf)', '% Electricity', '% Gas', '% Steam']])
print('-----2016----------------------------')
print(df2016_BU[['Site EUI (kBTU/sf)', '% Electricity', '% Gas', '% Steam']])
print('-----2017----------------------------')
print(df2017_BU[['Site EUI (kBTU/sf)', '% Electricity', '% Gas', '% Steam']])
print('----------------------------')

-----2015----------------------------
     Site EUI (kBTU/sf) % Electricity % Gas % Steam
52                131.7           77%   23%      0%
54                 68.6           38%   62%      0%
55                 92.5           30%   70%      0%
56                100.6           42%   58%      0%
58                 44.4           67%    0%      0%
63                141.3           28%   72%      0%
65                 99.7           22%   78%      0%
141               174.3           49%   51%      0%
192               126.4           46%   54%      0%
199                 161           32%   68%      0%
200               154.3           50%   50%      0%
365               155.9           48%   52%      0%
366               323.1           53%   47%      0%
367               236.1           48%   52%      0%
370               360.4           38%   61%      0%
371               279.6           45%   55%      0%
735               192.5            6%   94%      0%
739                74.7   

In [5]:
# Assignment of montly data 

# Convert Site EUI (kBTU/sf) to float
df2015_BU['Site EUI (kBTU/sf)'] = pd.to_numeric(df2015_BU['Site EUI (kBTU/sf)'], errors='coerce')
df2016_BU['Site EUI (kBTU/sf)'] = pd.to_numeric(df2016_BU['Site EUI (kBTU/sf)'], errors='coerce')
df2017_BU['Site EUI (kBTU/sf)'] = pd.to_numeric(df2017_BU['Site EUI (kBTU/sf)'], errors='coerce')

# print(type(df2015_BU.loc[52, '% Electricity']))
# print(type(df2017_BU.loc['1654', '% Electricity']))

# Convert '% Electricity', '% Gas' to float
# Remove % in 2015, 2017 (2016 does not have % at the end of string)

# print(df2015_BU['% Electricity'])

df2015_BU['% Electricity'] = df2015_BU['% Electricity'].str.replace('%', '')
df2017_BU['% Electricity'] = df2017_BU['% Electricity'].str.replace('%', '')

df2015_BU['% Gas'] = df2015_BU['% Gas'].str.replace('%', '')
df2017_BU['% Gas'] = df2017_BU['% Gas'].str.replace('%', '')

# Fill NaN with 0
df2015_BU[['% Electricity', '% Gas']] = df2015_BU[['% Electricity', '% Gas']].fillna(0)
df2016_BU[['% Electricity', '% Gas']] = df2016_BU[['% Electricity', '% Gas']].fillna(0)
df2017_BU[['% Electricity', '% Gas']] = df2017_BU[['% Electricity', '% Gas']].fillna(0)

# To float
df2015_BU['% Electricity'] = pd.to_numeric(df2015_BU['% Electricity'], errors='coerce') / 100
df2016_BU['% Electricity'] = pd.to_numeric(df2016_BU['% Electricity'], errors='coerce')
df2017_BU['% Electricity'] = pd.to_numeric(df2017_BU['% Electricity'], errors='coerce') / 100

df2015_BU['% Gas'] = pd.to_numeric(df2015_BU['% Gas'], errors='coerce') / 100
df2016_BU['% Gas'] = pd.to_numeric(df2016_BU['% Gas'], errors='coerce') 
df2017_BU['% Gas'] = pd.to_numeric(df2017_BU['% Gas'], errors='coerce') / 100


# Calculate yearly electricity and gas consumption (in kBTU/sf) based on 'Site EUI (kBTU/sf)'
df2015_BU['G_EUI'] = df2015_BU['Site EUI (kBTU/sf)'] * df2015_BU['% Gas']
df2015_BU['E_EUI'] = df2015_BU['Site EUI (kBTU/sf)'] * df2015_BU['% Electricity']

df2016_BU['G_EUI'] = df2016_BU['Site EUI (kBTU/sf)'] * df2016_BU['% Gas']
df2016_BU['E_EUI'] = df2016_BU['Site EUI (kBTU/sf)'] * df2016_BU['% Electricity']

df2017_BU['G_EUI'] = df2017_BU['Site EUI (kBTU/sf)'] * df2017_BU['% Gas']
df2017_BU['E_EUI'] = df2017_BU['Site EUI (kBTU/sf)'] * df2017_BU['% Electricity']


In [6]:
# Testing code (For debugging purpose)
print('-----2015----------------------------')
print(df2015_BU[['Site EUI (kBTU/sf)', '% Electricity', '% Gas', 'E_EUI', 'G_EUI', ]])
print('-----2016----------------------------')
print(df2016_BU[['Site EUI (kBTU/sf)', '% Electricity', '% Gas', 'E_EUI', 'G_EUI', ]])
print('-----2017----------------------------')
print(df2017_BU[['Site EUI (kBTU/sf)', '% Electricity', '% Gas', 'E_EUI', 'G_EUI', ]])
print('----------------------------')


-----2015----------------------------
      Site EUI (kBTU/sf)  % Electricity  % Gas    E_EUI    G_EUI
52                 131.7           0.77   0.23  101.409   30.291
54                  68.6           0.38   0.62   26.068   42.532
55                  92.5           0.30   0.70   27.750   64.750
56                 100.6           0.42   0.58   42.252   58.348
58                  44.4           0.67   0.00   29.748    0.000
63                 141.3           0.28   0.72   39.564  101.736
65                  99.7           0.22   0.78   21.934   77.766
141                174.3           0.49   0.51   85.407   88.893
192                126.4           0.46   0.54   58.144   68.256
199                161.0           0.32   0.68   51.520  109.480
200                154.3           0.50   0.50   77.150   77.150
365                155.9           0.48   0.52   74.832   81.068
366                323.1           0.53   0.47  171.243  151.857
367                236.1           0.48   0.52  113.

In [8]:
# START FROM HERE - Chengyu Deng Mar 19

#Monthly distribution of Gas EUI in each BU property in 2015
df2015_BU[['G_Jan']] = df2015_BU[['G_EUI']] * G_dstri_2015_Norm[0] 
df2015_BU[['G_Feb']] = df2015_BU[['G_EUI']] * G_dstri_2015_Norm[1] 
df2015_BU[['G_Mar']] = df2015_BU[['G_EUI']] * G_dstri_2015_Norm[2]
df2015_BU[['G_Apr']] = df2015_BU[['G_EUI']] * G_dstri_2015_Norm[3] 
df2015_BU[['G_May']] = df2015_BU[['G_EUI']] * G_dstri_2015_Norm[4]
df2015_BU[['G_Jun']] = df2015_BU[['G_EUI']] * G_dstri_2015_Norm[5]
df2015_BU[['G_Jul']] = df2015_BU[['G_EUI']] * G_dstri_2015_Norm[6]
df2015_BU[['G_Aug']] = df2015_BU[['G_EUI']] * G_dstri_2015_Norm[7]
df2015_BU[['G_Sep']] = df2015_BU[['G_EUI']] * G_dstri_2015_Norm[8]
df2015_BU[['G_Oct']] = df2015_BU[['G_EUI']] * G_dstri_2015_Norm[9]
df2015_BU[['G_Nov']] = df2015_BU[['G_EUI']] * G_dstri_2015_Norm[10] 
df2015_BU[['G_Dec']] = df2015_BU[['G_EUI']] * G_dstri_2015_Norm[11] 


#Monthly distribution of Electricity EUI in each BU property in 2015
df2015_BU[['E_Jan']] = df2015_BU[['E_EUI']] * E_dstri_2015_Norm[0] 
df2015_BU[['E_Feb']] = df2015_BU[['E_EUI']] * E_dstri_2015_Norm[1] 
df2015_BU[['E_Mar']] = df2015_BU[['E_EUI']] * E_dstri_2015_Norm[2]
df2015_BU[['E_Apr']] = df2015_BU[['E_EUI']] * E_dstri_2015_Norm[3] 
df2015_BU[['E_May']] = df2015_BU[['E_EUI']] * E_dstri_2015_Norm[4]
df2015_BU[['E_Jun']] = df2015_BU[['E_EUI']] * E_dstri_2015_Norm[5]
df2015_BU[['E_Jul']] = df2015_BU[['E_EUI']] * E_dstri_2015_Norm[6]
df2015_BU[['E_Aug']] = df2015_BU[['E_EUI']] * E_dstri_2015_Norm[7]
df2015_BU[['E_Sep']] = df2015_BU[['E_EUI']] * E_dstri_2015_Norm[8]
df2015_BU[['E_Oct']] = df2015_BU[['E_EUI']] * E_dstri_2015_Norm[9]
df2015_BU[['E_Nov']] = df2015_BU[['E_EUI']] * E_dstri_2015_Norm[10] 
df2015_BU[['E_Dec']] = df2015_BU[['E_EUI']] * E_dstri_2015_Norm[11] 


#Monthly distribution of Gas EUI in each BU property in 2016
df2016_BU[['G_Jan']] = df2016_BU[['G_EUI']] * G_dstri_2016_Norm[0] 
df2016_BU[['G_Feb']] = df2016_BU[['G_EUI']] * G_dstri_2016_Norm[1] 
df2016_BU[['G_Mar']] = df2016_BU[['G_EUI']] * G_dstri_2016_Norm[2]
df2016_BU[['G_Apr']] = df2016_BU[['G_EUI']] * G_dstri_2016_Norm[3] 
df2016_BU[['G_May']] = df2016_BU[['G_EUI']] * G_dstri_2016_Norm[4]
df2016_BU[['G_Jun']] = df2016_BU[['G_EUI']] * G_dstri_2016_Norm[5]
df2016_BU[['G_Jul']] = df2016_BU[['G_EUI']] * G_dstri_2016_Norm[6]
df2016_BU[['G_Aug']] = df2016_BU[['G_EUI']] * G_dstri_2016_Norm[7]
df2016_BU[['G_Sep']] = df2016_BU[['G_EUI']] * G_dstri_2016_Norm[8]
df2016_BU[['G_Oct']] = df2016_BU[['G_EUI']] * G_dstri_2016_Norm[9]
df2016_BU[['G_Nov']] = df2016_BU[['G_EUI']] * G_dstri_2016_Norm[10] 
df2016_BU[['G_Dec']] = df2016_BU[['G_EUI']] * G_dstri_2016_Norm[11] 

#Monthly distribution of Electricity EUI in each BU property in 2016
df2016_BU[['E_Jan']] = df2016_BU[['E_EUI']] * E_dstri_2016_Norm[0] 
df2016_BU[['E_Feb']] = df2016_BU[['E_EUI']] * E_dstri_2016_Norm[1] 
df2016_BU[['E_Mar']] = df2016_BU[['E_EUI']] * E_dstri_2016_Norm[2]
df2016_BU[['E_Apr']] = df2016_BU[['E_EUI']] * E_dstri_2016_Norm[3] 
df2016_BU[['E_May']] = df2016_BU[['E_EUI']] * E_dstri_2016_Norm[4]
df2016_BU[['E_Jun']] = df2016_BU[['E_EUI']] * E_dstri_2016_Norm[5]
df2016_BU[['E_Jul']] = df2016_BU[['E_EUI']] * E_dstri_2016_Norm[6]
df2016_BU[['E_Aug']] = df2016_BU[['E_EUI']] * E_dstri_2016_Norm[7]
df2016_BU[['E_Sep']] = df2016_BU[['E_EUI']] * E_dstri_2016_Norm[8]
df2016_BU[['E_Oct']] = df2016_BU[['E_EUI']] * E_dstri_2016_Norm[9]
df2016_BU[['E_Nov']] = df2016_BU[['E_EUI']] * E_dstri_2016_Norm[10] 
df2016_BU[['E_Dec']] = df2016_BU[['E_EUI']] * E_dstri_2016_Norm[11] 

#Monthly distribution of Gas EUI in each BU property in 2017
df2017_BU[['G_Jan']] = df2017_BU[['G_EUI']] * G_dstri_2017_Norm[0] 
df2017_BU[['G_Feb']] = df2017_BU[['G_EUI']] * G_dstri_2017_Norm[1] 
df2017_BU[['G_Mar']] = df2017_BU[['G_EUI']] * G_dstri_2017_Norm[2]
df2017_BU[['G_Apr']] = df2017_BU[['G_EUI']] * G_dstri_2017_Norm[3] 
df2017_BU[['G_May']] = df2017_BU[['G_EUI']] * G_dstri_2017_Norm[4]
df2017_BU[['G_Jun']] = df2017_BU[['G_EUI']] * G_dstri_2017_Norm[5]
df2017_BU[['G_Jul']] = df2017_BU[['G_EUI']] * G_dstri_2017_Norm[6]
df2017_BU[['G_Aug']] = df2017_BU[['G_EUI']] * G_dstri_2017_Norm[7]
df2017_BU[['G_Sep']] = df2017_BU[['G_EUI']] * G_dstri_2017_Norm[8]
df2017_BU[['G_Oct']] = df2017_BU[['G_EUI']] * G_dstri_2017_Norm[9]
df2017_BU[['G_Nov']] = df2017_BU[['G_EUI']] * G_dstri_2017_Norm[10] 
df2017_BU[['G_Dec']] = df2017_BU[['G_EUI']] * G_dstri_2017_Norm[11] 

#Monthly distribution of Electricity EUI in each BU property in 2017
df2017_BU[['E_Jan']] = df2017_BU[['E_EUI']] * E_dstri_2017_Norm[0] 
df2017_BU[['E_Feb']] = df2017_BU[['E_EUI']] * E_dstri_2017_Norm[1] 
df2017_BU[['E_Mar']] = df2017_BU[['E_EUI']] * E_dstri_2017_Norm[2]
df2017_BU[['E_Apr']] = df2017_BU[['E_EUI']] * E_dstri_2017_Norm[3] 
df2017_BU[['E_May']] = df2017_BU[['E_EUI']] * E_dstri_2017_Norm[4]
df2017_BU[['E_Jun']] = df2017_BU[['E_EUI']] * E_dstri_2017_Norm[5]
df2017_BU[['E_Jul']] = df2017_BU[['E_EUI']] * E_dstri_2017_Norm[6]
df2017_BU[['E_Aug']] = df2017_BU[['E_EUI']] * E_dstri_2017_Norm[7]
df2017_BU[['E_Sep']] = df2017_BU[['E_EUI']] * E_dstri_2017_Norm[8]
df2017_BU[['E_Oct']] = df2017_BU[['E_EUI']] * E_dstri_2017_Norm[9]
df2017_BU[['E_Nov']] = df2017_BU[['E_EUI']] * E_dstri_2017_Norm[10] 
df2017_BU[['E_Dec']] = df2017_BU[['E_EUI']] * E_dstri_2017_Norm[11] 

#For Testing 
print('--------------2015----------------------------')
print('Monthly distribution of gas EUI in BU properties in 2015')
print(df2015_BU[['G_Jan','G_Feb','G_Mar','G_Apr','G_May','G_Jun','G_Jul','G_Aug','G_Sep','G_Oct','G_Nov','G_Dec']])
print('Monthly distribution of electricity EUI in BU properties in 2015')
print(df2015_BU[['E_Jan','E_Feb','E_Mar','E_Apr','E_May','E_Jun','E_Jul','E_Aug','E_Sep','E_Oct','E_Nov','E_Dec']])
print('--------------2016----------------------------')
print('Monthly distribution of gas EUI in BU properties in 2015')
print(df2016_BU[['G_Jan','G_Feb','G_Mar','G_Apr','G_May','G_Jun','G_Jul','G_Aug','G_Sep','G_Oct','G_Nov','G_Dec']])
print('Monthly distribution of electricity EUI in BU properties in 2015')
print(df2016_BU[['E_Jan','E_Feb','E_Mar','E_Apr','E_May','E_Jun','E_Jul','E_Aug','E_Sep','E_Oct','E_Nov','E_Dec']])
print('--------------2017----------------------------')
print('Monthly distribution of gas EUI in BU properties in 2015')
print(df2017_BU[['G_Jan','G_Feb','G_Mar','G_Apr','G_May','G_Jun','G_Jul','G_Aug','G_Sep','G_Oct','G_Nov','G_Dec']])
print('Monthly distribution of electricity EUI in BU properties in 2015')
print(df2017_BU[['E_Jan','E_Feb','E_Mar','E_Apr','E_May','E_Jun','E_Jul','E_Aug','E_Sep','E_Oct','E_Nov','E_Dec']])

--------------2015----------------------------
Monthly distribution of gas EUI in BU properties in 2015
          G_Jan      G_Feb      G_Mar      G_Apr     G_May     G_Jun  \
52     7.581128   3.822416   2.558246   1.705684  1.238717  1.165051   
54    10.644764   5.367106   3.592068   2.394974  1.739300  1.635864   
55    16.205409   8.170791   5.468504   3.646069  2.647880  2.490411   
56    14.603139   7.362924   4.927819   3.285572  2.386078  2.244178   
58     0.000000   0.000000   0.000000   0.000000  0.000000  0.000000   
63    25.462139  12.838048   8.592181   5.728748  4.160382  3.912965   
65    19.463009   9.813278   6.567779   4.378999  3.180155  2.991032   
141   22.247837  11.217392   7.507517   5.005560  3.635182  3.418998   
192   17.082879   8.613213   5.764606   3.843491  2.791254  2.625259   
199   27.400281  13.815262   9.246206   6.164812  4.477065  4.210814   
200   19.308839   9.735545   6.515754   4.344312  3.154965  2.967339   
365   20.289423  10.229957   6.8

**Part 3 Visualization of monthly energy consumption**

Visualize the processed data and be ready to do further analysis. 

In [ ]:
# Data visualization

# Visualize:
# G_dstri_2015_Norm 
# E_dstri_2015_Norm 

# G_dstri_2016_Norm 
# E_dstri_2016_Norm 

# G_dstri_2017_Norm 
# E_dstri_2017_Norm 
